In [1]:
from data_loading import load_tumors
import matplotlib.pyplot as plt
import glob
import pandas as pd
import numpy as np
from PIL import Image
from keras.layers import Input, Embedding, LSTM, Dense, Conv2D
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tqdm import tqdm

import tensorflow as tf
from tensorflow.python.client import device_lib
print device_lib.list_local_devices()

IMAGE_HEIGHT, IMAGE_WIDTH = 240, 240
TUMOR_IMAGES = 155
df = pd.read_csv("data/survival_data.csv")
df.head()

Using TensorFlow backend.


[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2021656590346052878
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 10978538292
locality {
  bus_id: 1
}
incarnation: 17480562614891994532
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0"
]


,Brats17ID,Age,Survival
0,Brats17_TCIA_167_1,74.907,153
1,Brats17_TCIA_242_1,66.479,147
2,Brats17_TCIA_319_1,64.860,254
3,Brats17_TCIA_469_1,63.899,519
4,Brats17_TCIA_218_1,57.345,346


In [2]:
IMAGES_PATH = '/mnt/naruto/Seminar-Med-Seg2017/BRATS2017/HGG/Png-HGG/'
X = np.zeros((len(df), IMAGE_HEIGHT, IMAGE_WIDTH, TUMOR_IMAGES))

for index, row in tqdm(df.iterrows(), total=len(df)):
    # For every patient, fetch his tumor pictures and sort them properly
    glob_path = "{}{}_{}*nz*.png".format(IMAGES_PATH, row['Brats17ID'], 'seg')
    tumor_images = glob.glob(glob_path)
    tumor_images.sort(key=lambda x : int(x.split('.')[-2]))
    
    # Convert every tumor image to greyscale and add it to X
    for i in range(len(tumor_images)):
        img = Image.open(tumor_images[i]).convert('LA')
        img = np.asarray(img, dtype=np.uint8)[:, :, 0]
        X[index, :, :, i] = img
X.shape

100%|██████████| 163/163 [03:54<00:00,  1.43s/it]


(163, 240, 240, 155)

In [3]:
np.save("tumors_nz", X)

### 163 patients with tumor images of sizes 240x240 of which we have 155

In [4]:
Y = df['Survival']

In [8]:
with tf.device('/gpu:0'):
    
    main_input = Input(shape=X.shape[1:], dtype='float32', name='main_input')
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(main_input)
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    cnn_out = Dropout(0.25)(x)
    auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(cnn_out)
    
    auxiliary_input = Input(shape=(1,), name='aux_input')
    x = keras.layers.concatenate([cnn_out, auxiliary_input])
    x = Dense(64, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    main_output = Dense(1, activation='sigmoid', name='main_output')(x)
    
    
    model = Model(inputs=[main_input, auxiliary_input], outputs=[cnn_out, auxiliary_output])
    
    model.compile(optimizer='rmsprop',
              loss={'main_output': 'mean_squared_error', 'aux_output': 'mean_squared_error'},
              loss_weights={'main_output': 1., 'aux_output': 0.2})

    # And trained it via:
    model.fit({'main_input': X, 'aux_input': df['Age']},
              {'main_output': Y, 'aux_output': Y},
              epochs=50, batch_size=32)

ValueError: `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 60, 60, 64), (None, 1)]